In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
SEASON = '21'

In [3]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [4]:
from fastai.tabular.all import * 
from utils import *

In [5]:
path_raw = Path('raw_data')
path_data = Path('data')

In [6]:
!ls -lah -t {str(path_raw)}

total 15232
drwxr-xr-x  25 twtang  staff   800B Feb 25 22:50 21
-rw-r--r--   1 twtang  staff   703K Feb 25 05:54 data.zip
-rw-r--r--@  1 twtang  staff    10K Nov 21 10:01 .DS_Store
drwxr-xr-x  22 twtang  staff   704B Nov 20 20:04 ..
drwxr-xr-x  26 twtang  staff   832B Nov 20 19:55 .
drwx------@ 24 twtang  staff   768B Nov 20 19:55 13
drwx------@ 24 twtang  staff   768B Nov 20 19:55 12
drwx------@ 24 twtang  staff   768B Nov 20 19:54 11
drwx------@ 24 twtang  staff   768B Nov 20 19:53 14
drwx------@ 24 twtang  staff   768B Oct 30 14:35 20
drwx------@ 24 twtang  staff   768B Oct 30 14:35 19
drwx------@ 24 twtang  staff   768B Oct 30 14:35 18
drwx------@ 24 twtang  staff   768B Oct 30 14:35 17
drwx------@ 24 twtang  staff   768B Oct 30 14:35 16
drwx------@ 24 twtang  staff   768B Oct 30 14:34 15
-rw-r--r--   1 twtang  staff   314K Oct 29 06:22 21.zip
-rw-r--r--@  1 twtang  staff   615K Aug 25  2021 17.zip
-rw-r--r--@  1 twtang  staff   889K Jun 18  2021 19.zip
-rw-r--r--@  1 twtang  staff

In [7]:
sorted(list(path_raw.iterdir()))

[Path('raw_data/.DS_Store'),
 Path('raw_data/11'),
 Path('raw_data/11.zip'),
 Path('raw_data/12'),
 Path('raw_data/12.zip'),
 Path('raw_data/13'),
 Path('raw_data/13.zip'),
 Path('raw_data/14'),
 Path('raw_data/14.zip'),
 Path('raw_data/15'),
 Path('raw_data/15.zip'),
 Path('raw_data/16'),
 Path('raw_data/16.zip'),
 Path('raw_data/17'),
 Path('raw_data/17.zip'),
 Path('raw_data/18'),
 Path('raw_data/18.zip'),
 Path('raw_data/19'),
 Path('raw_data/19.zip'),
 Path('raw_data/20'),
 Path('raw_data/20.zip'),
 Path('raw_data/21'),
 Path('raw_data/21.zip'),
 Path('raw_data/data.zip')]

In [8]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [str(s).zfill(2) for s in range(11, 22)]

In [9]:
dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [10]:
df = pd.concat(dfs)
df = df.dropna()
df = df.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [11]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df, stats)

In [12]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df, stats)

In [13]:
df = df.merge(df_home, 'left', ['HomeTeam', 'Date']).merge(df_away, 'left', ['AwayTeam', 'Date'])

In [14]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df = df.merge(df_for.rename(columns=cols_home), 'left', ['HomeTeam', 'Date']).merge(df_for.rename(columns=cols_away), 'left', ['AwayTeam', 'Date'])

In [15]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df = df.merge(df_against.rename(columns=cols_home), 'left', ['HomeTeam', 'Date']).merge(df_against.rename(columns=cols_away), 'left', ['AwayTeam', 'Date'])

In [16]:
df = df.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [17]:
df.to_csv(path_data/'data.csv', index=False)

In [18]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HS', 'AS',
       'HST', 'AST', 'HC', 'AC', 'Season', 'FTHGLast5Avg', 'HSLast5Avg',
       'HSTLast5Avg', 'HCLast5Avg', 'FTAGLast5Avg', 'ASLast5Avg',
       'ASTLast5Avg', 'ACLast5Avg', 'HomeFTGForLast5Avg', 'HomeSForLast5Avg',
       'HomeSTForLast5Avg', 'HomeCForLast5Avg', 'AwayFTGForLast5Avg',
       'AwaySForLast5Avg', 'AwaySTForLast5Avg', 'AwayCForLast5Avg',
       'HomeFTGAgainstLast5Avg', 'HomeSAgainstLast5Avg',
       'HomeSTAgainstLast5Avg', 'HomeCAgainstLast5Avg',
       'AwayFTGAgainstLast5Avg', 'AwaySAgainstLast5Avg',
       'AwaySTAgainstLast5Avg', 'AwayCAgainstLast5Avg'],
      dtype='object')

In [19]:
df[['HC', 'AC', 'FTHGLast5Avg', 'HSLast5Avg',
    'HSTLast5Avg', 'HCLast5Avg', 'FTAGLast5Avg', 'ASLast5Avg',
    'ASTLast5Avg', 'ACLast5Avg', 'HomeFTGForLast5Avg', 'HomeSForLast5Avg',
    'HomeSTForLast5Avg', 'HomeCForLast5Avg', 'AwayFTGForLast5Avg',
    'AwaySForLast5Avg', 'AwaySTForLast5Avg', 'AwayCForLast5Avg',
    'HomeFTGAgainstLast5Avg', 'HomeSAgainstLast5Avg',
    'HomeSTAgainstLast5Avg', 'HomeCAgainstLast5Avg',
    'AwayFTGAgainstLast5Avg', 'AwaySAgainstLast5Avg',
    'AwaySTAgainstLast5Avg', 'AwayCAgainstLast5Avg']].corr()['HC']

HC                        1.000000
AC                       -0.210885
FTHGLast5Avg              0.114352
HSLast5Avg                0.129048
HSTLast5Avg               0.140832
HCLast5Avg                0.144876
FTAGLast5Avg             -0.081937
ASLast5Avg               -0.086712
ASTLast5Avg              -0.067156
ACLast5Avg               -0.060348
HomeFTGForLast5Avg        0.110966
HomeSForLast5Avg          0.132497
HomeSTForLast5Avg         0.142337
HomeCForLast5Avg          0.147932
AwayFTGForLast5Avg       -0.087375
AwaySForLast5Avg         -0.083471
AwaySTForLast5Avg        -0.069831
AwayCForLast5Avg         -0.060881
HomeFTGAgainstLast5Avg   -0.073377
HomeSAgainstLast5Avg     -0.086165
HomeSTAgainstLast5Avg    -0.055460
HomeCAgainstLast5Avg     -0.065052
AwayFTGAgainstLast5Avg    0.095905
AwaySAgainstLast5Avg      0.136328
AwaySTAgainstLast5Avg     0.130657
AwayCAgainstLast5Avg      0.140114
Name: HC, dtype: float64

In [20]:
df[['HC', 'AC', 'FTHGLast5Avg', 'HSLast5Avg',
    'HSTLast5Avg', 'HCLast5Avg', 'FTAGLast5Avg', 'ASLast5Avg',
    'ASTLast5Avg', 'ACLast5Avg', 'HomeFTGForLast5Avg', 'HomeSForLast5Avg',
    'HomeSTForLast5Avg', 'HomeCForLast5Avg', 'AwayFTGForLast5Avg',
    'AwaySForLast5Avg', 'AwaySTForLast5Avg', 'AwayCForLast5Avg',
    'HomeFTGAgainstLast5Avg', 'HomeSAgainstLast5Avg',
    'HomeSTAgainstLast5Avg', 'HomeCAgainstLast5Avg',
    'AwayFTGAgainstLast5Avg', 'AwaySAgainstLast5Avg',
    'AwaySTAgainstLast5Avg', 'AwayCAgainstLast5Avg']].corr()['AC']

HC                       -0.210885
AC                        1.000000
FTHGLast5Avg             -0.079428
HSLast5Avg               -0.083205
HSTLast5Avg              -0.071288
HCLast5Avg               -0.054713
FTAGLast5Avg              0.099261
ASLast5Avg                0.112269
ASTLast5Avg               0.121393
ACLast5Avg                0.131249
HomeFTGForLast5Avg       -0.073408
HomeSForLast5Avg         -0.086974
HomeSTForLast5Avg        -0.073537
HomeCForLast5Avg         -0.054978
AwayFTGForLast5Avg        0.100868
AwaySForLast5Avg          0.114218
AwaySTForLast5Avg         0.126291
AwayCForLast5Avg          0.142477
HomeFTGAgainstLast5Avg    0.088631
HomeSAgainstLast5Avg      0.117084
HomeSTAgainstLast5Avg     0.115293
HomeCAgainstLast5Avg      0.139489
AwayFTGAgainstLast5Avg   -0.073194
AwaySAgainstLast5Avg     -0.095394
AwaySTAgainstLast5Avg    -0.064437
AwayCAgainstLast5Avg     -0.058900
Name: AC, dtype: float64

## END